In [5]:
with open('dictionary.txt', 'r', encoding='utf8') as f:  # Abrimos diccionario
    dictionary = f.readlines()  # Metemos todo el diccionario en el programa


In [6]:
fails = []
dict1 = {}
for line in dictionary:
    t = line.split('\t', maxsplit=1)
    try:
        key = t[0]
        value = t[1].rstrip('\n')  # Introducimos el diccionario en una variable
        dict1[key] = value
    except IndexError as e:
        fails.append((line, str(e)))  # Guardamos la línea y el mensaje de error juntos
    except Exception as e:  # Capturamos cualquier otro error potencial
        fails.append((line, str(e)))

In [7]:
import psycopg2
import os

postgres_password = os.environ['DATABASE_PASSWORD']


# connection establishment
conn = psycopg2.connect(
database="interlinear_dictionary",
    user='postgres',
    password= postgres_password,
    host='localhost',
    port= '5432'
)

conn.autocommit = True

# Creating a cursor object
cursor = conn.cursor()


for palabra, traduccion in dict1.items():

    try:
        # Insertar palabra y traducción directamente en la tabla 'dictionary'
        cursor.execute("INSERT INTO dict (word, translation) VALUES (%s, %s)", (palabra, traduccion))
    except Exception as e:
        print(palabra, traduccion)
        print(f'Fallo al insertar palabra: {palabra} y traducción: {traduccion} por {e}')




conn.commit()
cursor.close()
conn.close()



1 1
Fallo al insertar palabra: 1 y traducción: 1 por el nuevo registro para la relación «dict» viola la restricción «check» «dict_translation_check»
DETAIL:  La fila que falla contiene (8, 1, 1, null).

2 2
Fallo al insertar palabra: 2 y traducción: 2 por el nuevo registro para la relación «dict» viola la restricción «check» «dict_translation_check»
DETAIL:  La fila que falla contiene (36, 2, 2, null).

13 13
Fallo al insertar palabra: 13 y traducción: 13 por el nuevo registro para la relación «dict» viola la restricción «check» «dict_translation_check»
DETAIL:  La fila que falla contiene (39, 13, 13, null).

21 21
Fallo al insertar palabra: 21 y traducción: 21 por el nuevo registro para la relación «dict» viola la restricción «check» «dict_translation_check»
DETAIL:  La fila que falla contiene (42, 21, 21, null).

31 31
Fallo al insertar palabra: 31 y traducción: 31 por el nuevo registro para la relación «dict» viola la restricción «check» «dict_translation_check»
DETAIL:  La fila que

In [9]:
import string

def unique_words(text):
    """
    Extrae palabras únicas de un texto dado.
    
    :param text: String con el texto del cual extraer las palabras únicas.
    :return: Set con palabras únicas.
    """
    # Traducción de todos los signos de puntuación a espacios y luego dividir en palabras
    table = str.maketrans(string.punctuation, ' ' * len(string.punctuation))  # Crea una tabla de traducción
    stripped_text = text.translate(table)  # Utiliza la tabla de traducción para reemplazar puntuación por espacios
    words = set(stripped_text.split())  # Divide el texto en palabras y las convierte en un conjunto para eliminar duplicados
    return words

# Suponiendo que tienes un texto 'sample_text'
sample_text = "Hello, world! This is a test. Hello, again! Hello Hello hello"

# Obtienes las palabras únicas
unique_words_set = unique_words(sample_text)

print(unique_words_set)


{'Hello', 'a', 'is', 'again', 'hello', 'test', 'This', 'world'}


In [10]:
from main import translate_words

In [11]:
from googletrans import Translator

def translate_words(missed_words, dict_words):
    """
    Traduce palabras que no se encontraron en la base de datos y las inserta en el diccionario dado.

    :param missed_words: list, palabras no encontradas en la base de datos.
    :param dict_words: dict, diccionario existente de palabras y traducciones.
    :return: dict, diccionario actualizado con las nuevas traducciones.
    """
    trans = Translator(service_urls=['translate.googleapis.com'])

    # Crear una lista de palabras no vacías.
    non_empty_words = [word for word in missed_words if word]
    new_words = {}

    for word in non_empty_words:
        try:
            translation = trans.translate(word, dest="es", src="de")
            dict_words[word] = translation.text 
            new_words[word] = translation.text
        except:
            new_words[word] = 'unknown'

        except Exception as e:
            print(f'Error al traducir palabras: {e}')

    return dict_words, new_words

In [14]:
miss = ['gings', 'bitte', 'gar', 'na', 'wars', 'godrics', 'machs', 'verbrannt']
dict = {}
translate_words(miss, dict)

TypeError: translate_words() missing 1 required positional argument: 'auth_key'

In [36]:
import requests
import json

def translate_words(missed_words, dict_words):
    """
    Traduce palabras que no se encontraron en la base de datos y las inserta en el diccionario dado.

    :param missed_words: list, palabras no encontradas en la base de datos.
    :param dict_words: dict, diccionario existente de palabras y traducciones.
    :param auth_key: str, tu clave de autenticación para la API de DeepL.
    :return: dict, diccionario actualizado con las nuevas traducciones.
    """
    url = "https://api-free.deepl.com/v2/translate"
    headers = {
        "Authorization": "DeepL-Auth-Key 8e8c4f85-61fe-1841-a424-3ec30e2dabe1:fx",
        "User-Agent": "YourApp/1.2.3",
        "Content-Type": "application/json"
    }
    
    new_words = {}

    for word in missed_words:
        if word:  # Continuar solo si la palabra no está vacía.
            body = {
                "text": [word],
                "source_lang": "DE",
                "target_lang": "ES"  # Asumiendo que deseas traducir al español.
            }

            response = requests.post(url, headers=headers, json=body)
            
            try:
                response.raise_for_status()  # Esto lanzará un error si la solicitud no fue exitosa.
                translation_result = response.json()
                translated_text = translation_result['translations'][0]['text']
                dict_words[word] = translated_text
                new_words[word] = translated_text
            except requests.exceptions.HTTPError as e:
                print(f'HTTP error occurred: {e}')
                new_words[word] = 'unknown'
            except Exception as e:
                print(f'Error al traducir palabras: {e}')
                new_words[word] = 'unknown'

    return dict_words, new_words


In [37]:
miss = ['bitte', 'gar', 'na', 'wars', 'godrics', 'machs', 'verbrannt']
dict = {}
one, two = translate_words(miss, dict)
print(one)
print(two)




{'bitte': 'por favor', 'gar': 'incluso', 'na': 'na', 'wars': 'wass', 'godrics': 'godrics', 'machs': 'hazlo', 'verbrannt': 'quemado'}
{'bitte': 'por favor', 'gar': 'incluso', 'na': 'na', 'wars': 'wass', 'godrics': 'godrics', 'machs': 'hazlo', 'verbrannt': 'quemado'}


In [32]:
url = "https://api-free.deepl.com/v2/translate"
headers = {
    "Authorization": "DeepL-Auth-Key 8e8c4f85-61fe-1841-a424-3ec30e2dabe1:fx",
    "User-Agent": "YourApp/1.2.3",
    "Content-Type": "application/json"
}

new_words = {}

for word in miss:
    if word:  # Continuar solo si la palabra no está vacía.
        body = {
            "text": [word],
            "source_lang": "DE",
            "target_lang": "ES"  # Asumiendo que deseas traducir al español.
        }

        response = requests.post(url, headers=headers, json=body)
        response.raise_for_status()  # Esto lanzará un error si la solicitud no fue exitosa.
        translation_result = response.json()
        translated_text = translation_result['translations'][0]['text']

        print(response)
        print(translation_result)
        print(f"word: {word} -> translation: {translated_text}")

<Response [200]>
{'translations': [{'detected_source_language': 'DE', 'text': 'por favor'}]}
word: bitte -> translation: por favor
<Response [200]>
{'translations': [{'detected_source_language': 'DE', 'text': 'incluso'}]}
word: gar -> translation: incluso
<Response [200]>
{'translations': [{'detected_source_language': 'DE', 'text': 'na'}]}
word: na -> translation: na
<Response [200]>
{'translations': [{'detected_source_language': 'DE', 'text': 'wass'}]}
word: wars -> translation: wass
<Response [200]>
{'translations': [{'detected_source_language': 'DE', 'text': 'godrics'}]}
word: godrics -> translation: godrics
<Response [200]>
{'translations': [{'detected_source_language': 'DE', 'text': 'hazlo'}]}
word: machs -> translation: hazlo
<Response [200]>
{'translations': [{'detected_source_language': 'DE', 'text': 'quemado'}]}
word: verbrannt -> translation: quemado


In [1]:
import os

# Reemplaza 'mi_variable' con el nombre de tu variable de entorno
variable = 

print(variable)


8e8c4f85-61fe-1841-a424-3ec30e2dabe1:fx
